<a href="https://colab.research.google.com/github/hildegard9847/pyscripts/blob/main/agendador.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install flask

In [ ]:
!pip install git+git://github.com/gunthercox/ChatterBot.git@master

In [ ]:
! pip install schedule

In [ ]:
import sqlite3
from ChatterBot import ChatBot
import schedule
import time
import datetime
from flask import Flask, request, render_template

app = Flask(__name__)

# Crie uma conexão com o banco de dados SQLite
conn = sqlite3.connect('agenda.db')
cursor = conn.cursor()

# Crie a tabela de agendamento (se ainda não existir)
cursor.execute('''
    CREATE TABLE IF NOT EXISTS lembretes (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        data_hora TEXT,
        mensagem TEXT
    )
''')
conn.commit()

# Crie uma instância do ChatBot
bot = ChatBot('MeuBot')

# Função para agendar um lembrete
def schedule_task(reminder_time, reminder_text):
    try:
        reminder_time = datetime.datetime.strptime(reminder_time, '%Y-%m-%d %H:%M')
        schedule.every().day.at(reminder_time.strftime('%H:%M')).do(send_reminder, reminder_text)

        # Insira o lembrete no banco de dados
        cursor.execute('INSERT INTO lembretes (data_hora, mensagem) VALUES (?, ?)', (reminder_time.strftime('%Y-%m-%d %H:%M'), reminder_text))
        conn.commit()

        print(f"Lembrete agendado para {reminder_time.strftime('%Y-%m-%d %H:%M')}")
    except ValueError:
        print("Formato de data e horário inválido. Use 'YYYY-MM-DD HH:MM'.")

# Função para enviar um lembrete
def send_reminder(reminder_text):
    print("Lembrete:", reminder_text)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/agendar', methods=['POST'])
def agendar():
    reminder_time = request.form.get('data_hora')
    reminder_text = request.form.get('mensagem')
    schedule_task(reminder_time, reminder_text)
    return 'Lembrete agendado com sucesso!'

if __name__ == "__main__":
    app.run()
